In [ ]:
"""
Script Functionality:
This script calculates the total, mean, minimum (non-zero), and maximum precipitation for Ghana in 2021 
using Earth Engine and precipitation data from the CHIRPS dataset. The results are exported as GeoTIFF files to Google Drive.

Input Parameters:
- Shapefile for Ghana (replace the path to match your local environment).
- CHIRPS dataset for daily precipitation in 2021.

Dependencies:
- Earth Engine Python API: `pip install earthengine-api`
- GeoPandas: `pip install geopandas`
- Folium (optional for visualization): `pip install folium`
- JSON (standard library)

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""

import ee  # Import the Earth Engine library
import geopandas as gpd  # Import GeoPandas for shapefile handling
import json  # Import JSON for converting GeoDataFrames to GeoJSON
import folium  # Import folium for map visualization (optional)

# Initialize Earth Engine (make sure you are authenticated)
ee.Initialize()

# Read the local Shapefile for Ghana (replace with the actual path to your shapefile)
shapefile_path = 'E:/ghana/Province/shape/ghana.shp'
gdf = gpd.read_file(shapefile_path)  # Load the shapefile using GeoPandas

# Convert Shapefile to GeoJSON format for compatibility with Earth Engine
geojson = json.loads(gdf.to_json())

# Import the GeoJSON into Earth Engine as a FeatureCollection (Area of Interest)
aoi = ee.FeatureCollection(geojson)

# Define the dataset (CHIRPS daily precipitation data) and the time range for 2021
dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
            .filter(ee.Filter.date('2021-01-01', '2021-12-31'))

# Select only the 'precipitation' band from the dataset
precipitation = dataset.select('precipitation')

# Calculate the total precipitation for the entire year by summing all daily precipitation values
total_precipitation = precipitation.sum()

# Calculate the mean (average) precipitation for the year
mean_precipitation = precipitation.mean()

# Define a function to mask out areas with zero precipitation for any given day
def mask_zero_values(image):
    """
    Masks out areas where the precipitation value is zero for a single day.
    
    Args:
        image (ee.Image): An image of daily precipitation values.

    Returns:
        ee.Image: The image with zero values masked.
    """
    return image.updateMask(image.gt(0))  # Keep only pixels where precipitation is greater than 0

# Apply the mask_zero_values function to the precipitation data
precipitation_non_zero = precipitation.map(mask_zero_values)

# Calculate the minimum precipitation values for non-zero areas, masking areas that never had precipitation
min_precipitation_non_zero = precipitation_non_zero.min()

# Unmask areas with no precipitation for the entire year, setting their values to 0
min_precipitation = min_precipitation_non_zero.unmask(0).where(total_precipitation.eq(0), 0)

# Calculate the maximum precipitation for the year (i.e., the highest daily value)
max_precipitation = precipitation.max()

# Define the output file names for exporting the data to Google Drive
output_total = 'total_precipitation_2021.tif'
output_mean = 'mean_precipitation_2021.tif'
output_min = 'min_precipitation_2021.tif'
output_max = 'max_precipitation_2021.tif'

# Export the total precipitation data to Google Drive as a GeoTIFF
task_total = ee.batch.Export.image.toDrive(
    image=total_precipitation.clip(aoi),  # Clip the image to the area of interest
    description='Total_Precipitation_2021',  # Description of the export task
    folder='GEE_Files',  # Google Drive folder to save the file
    fileNamePrefix=output_total.split('.')[0],  # File name without extension
    scale=5566,  # Spatial resolution in meters
    region=aoi.geometry().bounds().getInfo()['coordinates'],  # The geographic region to export
    fileFormat='GeoTIFF'  # File format
)

# Export the mean precipitation data to Google Drive
task_mean = ee.batch.Export.image.toDrive(
    image=mean_precipitation.clip(aoi),
    description='Mean_Precipitation_2021',
    folder='GEE_Files',
    fileNamePrefix=output_mean.split('.')[0],
    scale=5566,
    region=aoi.geometry().bounds().getInfo()['coordinates'],
    fileFormat='GeoTIFF'
)

# Export the minimum precipitation data to Google Drive
task_min = ee.batch.Export.image.toDrive(
    image=min_precipitation.clip(aoi),
    description='Min_Precipitation_2021',
    folder='GEE_Files',
    fileNamePrefix=output_min.split('.')[0],
    scale=5566,
    region=aoi.geometry().bounds().getInfo()['coordinates'],
    fileFormat='GeoTIFF'
)

# Export the maximum precipitation data to Google Drive
task_max = ee.batch.Export.image.toDrive(
    image=max_precipitation.clip(aoi),
    description='Max_Precipitation_2021',
    folder='GEE_Files',
    fileNamePrefix=output_max.split('.')[0],
    scale=5566,
    region=aoi.geometry().bounds().getInfo()['coordinates'],
    fileFormat='GeoTIFF'
)

# Start the export tasks (initiate the export process to Google Drive)
task_total.start()
task_mean.start()
task_min.start()
task_max.start()

# Notify that the export tasks have been started
print('Export tasks have been started. Check your Google Drive for the results.')

# End of the script


In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Zeshuo Li (1023172917@njupt.edu.cn)
Date: October 1 2024
This code is protected by copyright and may not be copied or distributed without permission.

Script Functionality:
This script resamples a precipitation GeoTIFF raster to match a given template raster, ensuring alignment in resolution, 
affine transform, and coordinate reference system (CRS). The output is saved as a resampled GeoTIFF file.

Input Parameters:
- Template raster file (providing the target resolution and CRS).
- Precipitation raster file (to be resampled).
- Output path for the resampled raster.

Dependencies:
- Rasterio: `pip install rasterio`

Version Information:
- Version: v1.0
- Change Log: October 1 2024.
"""

import rasterio  # Import Rasterio for raster handling
from rasterio.enums import Resampling  # Import Resampling methods from Rasterio
from rasterio.warp import reproject, calculate_default_transform  # Import functions for reprojection

# Path settings (adjust these paths to match your local environment)
template_raster_path = 'E:/ghana/Raster_Templates/ghana_template/0.01.tif'  # Path to the raster template file
precipitation_raster_path = 'E:/ghana/Precipitation_data/precipitation/total_precipitation_2022.tif'  # Path to the exported precipitation data
output_resampled_path = 'E:/ghana/Precipitation_data/precipitation/resampling/0.01/2022/total_precipitation_2022_0.01.tif'  # Path for saving the resampled result

# Read the template raster to extract geographic metadata (for alignment and resolution)
with rasterio.open(template_raster_path) as template_raster:
    template_meta = template_raster.meta.copy()  # Copy the metadata from the template
    template_transform = template_raster.transform  # Get the affine transform from the template
    template_crs = template_raster.crs  # Get the Coordinate Reference System (CRS) from the template
    template_resolution = (template_raster.res[0], template_raster.res[1])  # Get the resolution of the template

# Read the precipitation raster that needs to be resampled
with rasterio.open(precipitation_raster_path) as src:
    src_transform = src.transform  # Get the affine transform from the precipitation raster
    src_crs = src.crs  # Get the CRS of the precipitation raster
    src_resolution = (src.res[0], src.res[1])  # Get the resolution of the precipitation raster

    # Create new metadata for the output file, adjusting it to match the template raster
    out_meta = src.meta.copy()  # Copy metadata from the source raster
    out_meta.update({
        'crs': template_crs,  # Use the CRS from the template (WGS84, EPSG:4326)
        'transform': template_transform,  # Use the affine transform from the template (for alignment)
        'width': template_raster.width,  # Set output width to match the template raster
        'height': template_raster.height,  # Set output height to match the template raster
        'driver': 'GTiff'  # Output format set to GeoTIFF
    })

    # Open a new file to write the resampled raster data
    with rasterio.open(output_resampled_path, 'w', **out_meta) as dst:
        # Iterate through each band of the precipitation raster to resample it
        for i in range(1, src.count + 1):  # Loop through all raster bands
            reproject(
                source=rasterio.band(src, i),  # Source band from the precipitation raster
                destination=rasterio.band(dst, i),  # Destination band in the new file
                src_transform=src_transform,  # Source raster's affine transform
                src_crs=src_crs,  # Source CRS (coordinate system)
                dst_transform=template_transform,  # Use the affine transform from the template
                dst_crs=template_crs,  # Target CRS (WGS84)
                resampling=Resampling.bilinear  # Resample using bilinear interpolation
            )

# Print message to indicate successful resampling
print(f'Resampled and aligned data saved to: {output_resampled_path}')

# End of the script



